In [12]:
import time
import numpy as np
import math
from multiprocessing.pool import ThreadPool as Pool
from numpy.linalg import inv, pinv

In [20]:
def divide_into_blocks(matrix: np.ndarray, *args, verbose=True):
    """
    1. You have to specify what *args can be for statically typed.
    2. Instead of setting blocks_amount to None, we can let it be empty.
    """

    msize = matrix.shape
    # if verbose is True:
    #     print("Block Division")
    #     print("Matrix size for binv: ")
    #     print(msize)

    blocks_amount = None if not args else args[0]
    if blocks_amount is None or blocks_amount == 0:
        blocks_amount = msize[1]
    block_size = msize[1] // blocks_amount + (msize[1] % blocks_amount)
    blocks = {}
    start = 0

    # Loops
    for b in range(blocks_amount + 1):
        end = (start + block_size) - 1
        if block_size == 1:
            end = start + 1
        if end > msize[1] - 1:
            end = msize[1]

        block = matrix[:, start:end]
        if block.size != 0:
            blocks[b] = block
        start = end
    return blocks


def pinvblock(block: np.ndarray, block_id: int, pblocks: dict):
    pinv_block = pinv(block)
    pblocks[block_id] = pinv_block
    return pblocks


def pinvblocks(blocks: dict) -> dict:
    pinvblocks = {}
    pool = Pool(len(blocks))
    for block_id, block in blocks.items():
        pool.apply_async(pinvblock, (block, block_id, pinvblocks))
    pool.close()
    pool.join()
    return pinvblocks


# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# C++ potentially
def form_row(pblock_id: int, pblock: np.ndarray, blocks: dict, rows: dict):
    """
    The concatenation in the end can be improved because it always allocates new
    memory each time it is called. Don't we know the number of rows
    in the beginning?
    """

    row = []
    for i in range(len(blocks)):
        if i == pblock_id:
            result = np.eye(pblock.shape[0])
        else:
            result = np.matmul(pblock, blocks.get(i))
        # print("result ", type(result), result.shape)
        row.append(result)

    # Optimize
    rows[pblock_id] = np.concatenate((row), axis=1)
    # print("Final result: ", rows.shape)

    return rows


def build_Pmatrix(blocks: dict, pblocks: dict) -> np.ndarray:
    """
    Nr of blocks: Make each block as small as possible
    Concenate by allocation before.
    Do we parralelize along dim0 or dim1?
    """
    result_rows = {}
    pool = Pool(len(blocks))
    for id, pblock in pblocks.items():
        pool.apply_async(form_row, (id, pblock, blocks, result_rows))
    pool.close()
    pool.join()
    sorted(result_rows)
    rows = list(result_rows.values())
    return np.concatenate(rows, axis=0)


def binv(matrix: np.ndarray, *args, verbose=True):
    blocks = divide_into_blocks(matrix, *args)
    pblocks = pinvblocks(blocks)
    Pmatrix = build_Pmatrix(blocks, pblocks)
    inverted_Pmatrix = inv(Pmatrix)
    sorted(pblocks)
    pseudoinversed = np.matmul(
        inverted_Pmatrix, np.concatenate(list(pblocks.values()), axis=0)
    )
    return pseudoinversed

In [21]:
from numpy.linalg import inv, norm


def fast_binv(A: np.array, verbose=True):
    binvR_array = np.empty(shape=(A.shape[1], A.shape[0]))
    for i in range(A.shape[1]):
        binvR_array[i, :] = (A[:, i] / norm(A[:, i])).T
    inv_matr = np.matmul(binvR_array, A)
    inv_matr = inv(inv_matr)
    result = np.matmul(inv_matr, binvR_array)
    return result

In [22]:
def hpinv(A: np.array):
    return np.matmul(np.linalg.inv(np.matmul(A.T, A)), A.T)

In [ ]:
import numpy as np


def super_fast_binv(C, split_index=None, method="svd"):
    """
    TODO: optimize! 
    
    Optimized pseudoinverse of [A | B] using QR/SVD block preprocessing.

    Parameters:
        C (ndarray): Input matrix of shape (m, n1 + n2)
        split_index (int or None): Index to split C into A and B (columns). If None, split in half.
        method (str): 'svd' (default) or 'qr'

    Returns:
        AB_pinv (ndarray): Pseudoinverse of the concatenated matrix [A | B]
    """
    m, n = C.shape
    if split_index is None:
        split_index = n // 2

    A = C[:, :split_index]
    B = C[:, split_index:]

    n1 = A.shape[1]
    n2 = B.shape[1]

    if method == "svd":
        UA, SA, VA_T = np.linalg.svd(A, full_matrices=False)
        UB, SB, VB_T = np.linalg.svd(B, full_matrices=False)

        RA_inv = VA_T.T * (1.0 / SA)
        RB_inv = VB_T.T * (1.0 / SB)

        # Pad the R blocks to same column size
        RA_inv_padded = np.hstack((RA_inv, np.zeros((n1, n2))))
        RB_inv_padded = np.hstack((np.zeros((n2, n1)), RB_inv))

    elif method == "qr":
        UA, RA = np.linalg.qr(A, mode="reduced")
        UB, RB = np.linalg.qr(B, mode="reduced")

        RA_inv = np.linalg.inv(RA)
        RB_inv = np.linalg.inv(RB)

        RA_inv_padded = np.hstack((RA_inv, np.zeros((n1, n2))))
        RB_inv_padded = np.hstack((np.zeros((n2, n1)), RB_inv))

    else:
        raise ValueError("Method must be 'svd' or 'qr'.")

    Q_comb = np.hstack((UA, UB))
    R_comb = np.vstack((RA_inv_padded, RB_inv_padded))  # shape (n1 + n2, n1 + n2)

    AB_pinv = R_comb @ Q_comb.T  # Final pseudoinverse
    return AB_pinv

### Description
*binv(matrix: np.ndarray, \*args)* -  the function which returns the Moore-Penrouse pseudo-inversed matrix for full column rank case using block inversion (the formula requires $A^+A=E$). \
Let $A=[A_1 A_2 \dots A_n]$ - full column rank (block) matrix, then the pseudoinversed matrix can be calculated by 
$$A^+ = [A_1 A_2 \dots A_n]^+=\begin{bmatrix}
        \begin{bmatrix}
            A_1^+ \\
            A_2^+ \\
            \dots \\
            A_n^+ \\
        \end{bmatrix}
        [A_1 A_2 \dots A_n]
    \end{bmatrix}^{-1}      \begin{bmatrix}
            A_1^+ \\
            A_2^+ \\
            \dots \\
            A_n^+ \\
        \end{bmatrix}.$$
        

### Usage
By default *binv(matrix: np.ndarray)* receives only matrix. Then it counts that each column is a different block. In that case, there is no advantage and the time of pseudoinversion is equal to the standard function *pinv*. \
**!** For effective calculation pass the amount of blocks in matrix *binv(matrix: np.ndarray, blocks_amount: int)*. Then each block will be calculated in a different thread. 
### Examples
    - binv(np.random.rand(rows, coulmns)) 
    - binv(np.random.rand(rows, columns), 10)
**Note.** To satisfy the full column rank condition there should be $ rows \geq columns$